## Pyo library tests : generating sounds from nothing

In [1]:
from pyo import *

In [3]:
s = Server().boot()
s.start()
# Defines tables for the amplitude, the ratio and the modulation index.
amp_table = CosTable([(0,0), (100,1), (1024,.5), (7000,.5), (8192,0)])
rat_table = ExpTable([(0, .5), (1500, .5), (2000, .25), (3500, .25),
                      (4000, 1), (5500, 1), (6000, .5), (8192, .5)])
ind_table = LinTable([(0, 20), (512, 10), (8192, 0)])

# call their graph() method. Use the "yrange" argument to set the minimum
# and maximum bundaries of the graph (defaults to 0 and 1).
amp_table.graph(title="Amplitude envelope")
rat_table.graph(title="Ratio envelope")
ind_table.graph(yrange=(0, 20), title="Modulation index envelope")

# Initialize the table readers (TableRead.play() must be called explicitly).
amp = TableRead(table=amp_table, freq=1, loop=False, mul=.3)
rat = TableRead(table=rat_table, freq=1, loop=False)
ind = TableRead(table=ind_table, freq=1, loop=False)

# Use the signals from the table readers to control an FM synthesis.
fm = FM(carrier=[100,100], ratio=rat, index=ind, mul=amp).out()

# Call the "note" function to generate an event.
def note(freq=100, dur=1):
    fm.carrier = [freq, freq*1.005]
    amp.freq = 1.0 / dur
    rat.freq = 1.0 / dur
    ind.freq = 1.0 / dur
    amp.play()
    rat.play()
    ind.play()

# Play one note, carrier = 100 Hz, duration = 2 seconds.
note(200, 2)

In [2]:
s = Server().boot()
s.start()
#amp_table = CosTable([(0,0), (100,1), (1024,.5), (7000,.5), (8192,0)]) #good
#amp_table = CurveTable([(-1000,0),(-148,0),(96,0),(1326,0),(6144,0)], 1, 10) #not very different from CosTable
amp_table = ExpTable([(0,0),(4096,1),(8192,0)], exp=2, inverse=True) #gives more of a glissando feeling
#amp_table = HannTable() #meh
#amp_table = HarmTable([1,0,.33,0,.2,0,.143,0,.111]) #meh
#amp_table = LinTable([(0,0), (100,1), (1000,.25), (8191,0)]) #accents
#amp_table = LogTable([(0,0), (2096, 2), (4095,1), (8192,0)]) #glissando with cuts
#amp_table = ParaTable() #not a lot of nuance --> weirdly brought deeper sounds ??
#amp_table = PartialTable([(1,1), (2.37, 0.5), (6.66, 0.4), (14.55, 0.3)]).normalize() #some kind of polyphony
#amp_table = SawTable(order=100)#.normalize() #not normalization brings more variation in amplitude which makes it interesting
#amp_table = SincTable(freq=math.pi*10, windowed=True) #brings a less musical but more interesting sound
#amp_table = SquareTable(order=(50))#.normalize() #order > 100 brings some kind of metronome
#amp_table = WinTable(type=1) #1 sounds like a bumblebee, 3 is more fluid than 2, 

freq = Xnoise(dist=6, freq=8, x1=10, mul=0.1, add=150) #common noise distribs 6, 4 is a bit soft, 10 varies more
#freq = RandInt(max=-10, freq=8, mul=100, add=1000) #max=-10 sounds good, too high can get uncomfortable for the ears
#freq = Randh(100, 1000, 8)
#freq = Randi(500, 3000, 4)

dur = XnoiseDur(dist="expon_min", min=[.05,0.5], max=[.9,1.5], x1=3)
amp = TableRead(table=amp_table, freq=dur, loop=True, mul=1).play()

#a = SineLoop(freq, feedback=0.01, mul=amp).out()
#a = Blit(freq=[100, 99.7]*freq, harms=dur, mul=amp).out() #noisy
#a = ChenLee(pitch=freq, chaos=dur, stereo=True, mul=amp, add=.5).out() #sounds like it's about to explode but very interesting
#a = CrossFM(carrier=freq, ratio=dur, ind1=2, ind2=2, mul=amp).out() #this one is fun
#a = FM(carrier=freq, ratio=dur, index=amp, mul=.2).out() #sounds like bells
#a = LFO(freq=freq, sharp=1, type=6, mul=amp, add=dur).out() #6 sounds like a UFO
#a = Lorenz(pitch=freq, mul=dur, add=amp).out() #sounds boring quickly
#a = Phasor(freq=freq, mul=dur, add=amp).out() #nice, good variation
#a = RCOsc(freq=freq, sharp=1, mul=dur, add=amp).out() #video game like
a = SuperSaw(freq=freq, detune=0.4, bal=0.7, mul=dur, add=amp).out()

In [3]:
s.stop()